## LSTM 准备-- 整个导图

### 将时间序列转换为监督学习

本节分外三步：

1. 将时间序列转化为监督学习问题。
2. 转化时间序列数据使其呈静态。
3. 转化观察值使其处于特定区间

In [10]:
from pandas import DataFrame
from pandas import  concat
from pandas import read_csv
from pandas import datetime

In [2]:
def parser(x):
    '''1年1月 转换成 1901-1-1'''
    return datetime.strptime('190'+x,'%Y年%m月')

In [4]:
series = read_csv('data/sales2.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [45]:
X = series.values

In [50]:
def timeseries_to_supervised(data, lag=1):
#     narray 转换成 DataFrame
    df = DataFrame(data)
#     往下移动一位
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
#     将list的两个原始拼接成两列
    df = concat(columns, axis=1)
#     将NaN 转换成 0
    df.fillna(0, inplace=True)
    return df

In [51]:
supervised = timeseries_to_supervised(X,1)

In [53]:
supervised.head()

,0,0
0,0.0,266.0
1,266.0,145.9
2,145.9,183.1
3,183.1,119.3
4,119.3,180.3


### 将时间序列转换为静态

时间是递增的，需要去掉时间的递增趋势。    
差分diff操作溢出趋势    

In [54]:
from pandas import Series

自定义差分

In [56]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        diff.append(dataset[i]-dataset[i-interval])
    return Series(diff)

差分恢复

In [57]:
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [59]:
series = read_csv('data/sales2.csv',header=0,parse_dates=[0], index_col=0, squeeze=True,date_parser=parser)

In [61]:
series.head()

Month
1901-01-01    266.0
1901-02-01    145.9
1901-03-01    183.1
1901-04-01    119.3
1901-05-01    180.3
Name: Sales of shampoo over a three year period, dtype: float64

In [63]:
inverted = list()
differenced = difference(series,1)
for i in range(len(differenced)):
    value = inverse_difference(series, differenced[i], len(series)-i)
    inverted.append(value)

    
inverted = Series(inverted)

### 转化时间序列使其处于特定区间

使用MinMaxScaler class转化数据集使其处在 [-1, 1] 区间内。和其他scikit-learn转换模块一样，它需要提供行列矩阵形式的数据。因此，我们必须在转换数据集之前变换NumPy数组。

In [69]:
X = series.values


In [70]:
X = X.reshape(len(X),1)


In [71]:
from sklearn.preprocessing import MinMaxScaler

In [84]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(X)
scaled_X = scaler.transform(X)
scaled_series = Series(scaled_X[:,0])
scaled_series.head()

0   -0.478585
1   -0.905456
2   -0.773236
3   -1.000000
4   -0.783188
dtype: float64

In [85]:
inverted_X = scalar.inverse_transform(scaled_X)
inverted_series = Series(inverted_X[:,0])
inverted_series.head()

0    266.0
1    145.9
2    183.1
3    119.3
4    180.3
dtype: float64

## LSTM 模型开发

`stateful` 保存状态， reset_states()清除状态    

LSTM层要求输入值须位于有维度的矩阵中；【例子、时间步、特征】。

**例子**：是指定义域中的独立观察值，通常是几列数据。    
**时间步**：是指特定观察值的给定变量的单独时间步。    
**特征**：是指的观察时得到的单独指标。      

依次为输入的第一维，第二维，第三维    
